In [1]:
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Bidirectional
import joblib

/home/quizhpilema.134643/anaconda3/envs/py38ml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-09-30 14:32:30.130468: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-30 14:32:30.166253: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-30 14:32:30.167108: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 14:32:30.892746: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [46]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU is available")
else:
    print("GPU is not available")
    
TRAIN_TEST_SPLIT_NUM = 0.05

GPU is not available


In [2]:
# Se lee la base de datos completa
data_river_2 = pd.read_csv('db_21.csv')
# Se asisgna la columna fecha como indice
data_river_2 = data_river_2.set_index('Fecha')
# Asignar un formato de fecha a la columna fecha
data_river_2.index = pd.to_datetime(data_river_2.index, format='%Y-%m-%d %H:%M:%S')

date_init = '1999-03-16 17:00'
date_end = '2023-09-30 23:00'

aitsu_pred = data_river_2[date_init:date_end] 
#Eliminacion de variables repetidas
aitsu_pred = aitsu_pred.drop(columns=['LLUVIA_Aitzu-Urola','NIVEL_RIO_Aitzu-Urola','CAUDAL_RIO_Aitzu-Urola'])
aitsu_pred

,LLUVIA_Aitzu,NIVEL_RIO_Aitzu,CAUDAL_RIO_Aitzu,LLUVIA_Aizarnazabal,NIVEL_RIO_Aizarnazabal,CAUDAL_RIO_Aizarnazabal,LLUVIA_Antzuola,LLUVIA_Arantzazu,LLUVIA_Araotz,LLUVIA_Arkaka,...,LLUVIA_Lareo,LLUVIA_Lastur,LLUVIA_Laurgain,LLUVIA_Matxinbenta,NIVEL_RIO_Matxinbenta,CAUDAL_RIO_Matxinbenta,LLUVIA_Troya,LLUVIA_zumarraga,LLUVIA_azpeitia,LLUVIA_legazpi
Fecha,,,,,,,,,,,,,,,,,,,,,
1999-03-16 17:00:00,NaN,0.260,1.1905,0.0,0.528,6.7375,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1999-03-16 18:00:00,NaN,0.260,1.1905,0.0,0.528,6.7375,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1999-03-16 19:00:00,NaN,0.260,1.1905,0.0,0.528,6.7375,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1999-03-16 20:00:00,NaN,0.260,1.1905,0.0,0.528,6.7375,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1999-03-16 21:00:00,NaN,0.260,1.1905,0.0,0.524,6.6102,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-30 19:00:00,0.0,0.120,0.1553,0.0,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.071,0.0218,0.0,0.0,0.0,0.0
2023-09-30 20:00:00,0.0,0.120,0.1553,0.0,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.071,0.0218,0.0,0.0,0.0,0.0
2023-09-30 21:00:00,0.0,0.118,0.1489,0.0,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.071,0.0218,0.0,0.0,0.0,0.0


In [3]:
df_sin_CAUDAL = aitsu_pred.drop(columns=['CAUDAL_RIO_Aitzu'])

In [4]:
#StandardScaler
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_sin_CAUDAL)
df_scaled = pd.DataFrame(df_scaled, columns=df_sin_CAUDAL.columns, index=df_sin_CAUDAL.index)
df_scaled

,LLUVIA_Aitzu,NIVEL_RIO_Aitzu,LLUVIA_Aizarnazabal,NIVEL_RIO_Aizarnazabal,CAUDAL_RIO_Aizarnazabal,LLUVIA_Antzuola,LLUVIA_Arantzazu,LLUVIA_Araotz,LLUVIA_Arkaka,LLUVIA_Barrendiola,...,LLUVIA_Lareo,LLUVIA_Lastur,LLUVIA_Laurgain,LLUVIA_Matxinbenta,NIVEL_RIO_Matxinbenta,CAUDAL_RIO_Matxinbenta,LLUVIA_Troya,LLUVIA_zumarraga,LLUVIA_azpeitia,LLUVIA_legazpi
Fecha,,,,,,,,,,,,,,,,,,,,,
1999-03-16 17:00:00,NaN,0.254916,-0.163239,0.282931,-0.018720,NaN,-0.172799,NaN,-0.175859,NaN,...,-0.197428,-0.177951,-0.164368,NaN,NaN,NaN,-0.16558,NaN,NaN,NaN
1999-03-16 18:00:00,NaN,0.254916,-0.163239,0.282931,-0.018720,NaN,-0.172799,NaN,-0.175859,NaN,...,-0.197428,-0.177951,-0.164368,NaN,NaN,NaN,-0.16558,NaN,NaN,NaN
1999-03-16 19:00:00,NaN,0.254916,-0.163239,0.282931,-0.018720,NaN,-0.172799,NaN,-0.175859,NaN,...,-0.197428,-0.177951,-0.164368,NaN,NaN,NaN,-0.16558,NaN,NaN,NaN
1999-03-16 20:00:00,NaN,0.254916,-0.163239,0.282931,-0.018720,NaN,-0.172799,NaN,-0.175859,NaN,...,-0.197428,-0.177951,-0.164368,NaN,NaN,NaN,-0.16558,NaN,NaN,NaN
1999-03-16 21:00:00,NaN,0.254916,-0.163239,0.265546,-0.029147,NaN,-0.172799,NaN,-0.175859,NaN,...,-0.197428,-0.177951,-0.164368,NaN,NaN,NaN,-0.16558,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-30 19:00:00,-0.18246,-0.662379,-0.163239,NaN,NaN,-0.176252,-0.172799,-0.177317,-0.175859,-0.198847,...,-0.197428,-0.177951,-0.164368,-0.1841,-0.7878,-0.434755,-0.16558,-0.158125,-0.110789,-0.178171
2023-09-30 20:00:00,-0.18246,-0.662379,-0.163239,NaN,NaN,-0.176252,-0.172799,-0.177317,-0.175859,-0.198847,...,-0.197428,-0.177951,-0.164368,-0.1841,-0.7878,-0.434755,-0.16558,-0.158125,-0.110789,-0.178171
2023-09-30 21:00:00,-0.18246,-0.675483,-0.163239,NaN,NaN,-0.176252,-0.172799,-0.177317,-0.175859,-0.198847,...,-0.197428,-0.177951,-0.164368,-0.1841,-0.7878,-0.434755,-0.16558,-0.158125,-0.110789,-0.178171


In [5]:
df_scaled.min()

LLUVIA_Aitzu                     -0.182460
NIVEL_RIO_Aitzu                  -1.238964
LLUVIA_Aizarnazabal              -0.163239
NIVEL_RIO_Aizarnazabal           -1.911868
CAUDAL_RIO_Aizarnazabal          -0.569588
LLUVIA_Antzuola                  -0.176252
LLUVIA_Arantzazu                 -0.172799
LLUVIA_Araotz                    -0.177317
LLUVIA_Arkaka                    -0.175859
LLUVIA_Barrendiola               -0.198847
NIVEL_RIO_Barrendiola            -0.973715
CAUDAL_RIO_Barrendiola           -0.602765
LLUVIA_Barrendiola_Pluviometro   -0.187596
LLUVIA_Elosua                    -0.183014
LLUVIA_Erdoizta                  -0.181858
LLUVIA_Ibaieder                  -0.172504
NIVEL_RIO_Ibaieder               -1.313843
CAUDAL_RIO_Ibaieder              -0.486235
LLUVIA_Lareo                     -0.197428
LLUVIA_Lastur                    -0.177951
LLUVIA_Laurgain                  -0.164368
LLUVIA_Matxinbenta               -0.184100
NIVEL_RIO_Matxinbenta            -1.483216
CAUDAL_RIO_

In [6]:
df_scaled.max()

LLUVIA_Aitzu                       74.828666
NIVEL_RIO_Aitzu                    10.332056
LLUVIA_Aizarnazabal                76.895763
NIVEL_RIO_Aizarnazabal             14.051492
CAUDAL_RIO_Aizarnazabal            37.073972
LLUVIA_Antzuola                    74.863316
LLUVIA_Arantzazu                   88.151622
LLUVIA_Araotz                     121.878846
LLUVIA_Arkaka                      86.035087
LLUVIA_Barrendiola                 56.032029
NIVEL_RIO_Barrendiola               9.302611
CAUDAL_RIO_Barrendiola             14.091113
LLUVIA_Barrendiola_Pluviometro     75.954298
LLUVIA_Elosua                      91.332722
LLUVIA_Erdoizta                    57.753400
LLUVIA_Ibaieder                    70.905043
NIVEL_RIO_Ibaieder                  9.599425
CAUDAL_RIO_Ibaieder                19.608274
LLUVIA_Lareo                      100.760898
LLUVIA_Lastur                      95.777019
LLUVIA_Laurgain                   116.160963
LLUVIA_Matxinbenta                 83.776757
NIVEL_RIO_

In [8]:
df = pd.DataFrame(aitsu_pred['CAUDAL_RIO_Aitzu']).merge(df_scaled, on='Fecha')
df

,CAUDAL_RIO_Aitzu,LLUVIA_Aitzu,NIVEL_RIO_Aitzu,LLUVIA_Aizarnazabal,NIVEL_RIO_Aizarnazabal,CAUDAL_RIO_Aizarnazabal,LLUVIA_Antzuola,LLUVIA_Arantzazu,LLUVIA_Araotz,LLUVIA_Arkaka,...,LLUVIA_Lareo,LLUVIA_Lastur,LLUVIA_Laurgain,LLUVIA_Matxinbenta,NIVEL_RIO_Matxinbenta,CAUDAL_RIO_Matxinbenta,LLUVIA_Troya,LLUVIA_zumarraga,LLUVIA_azpeitia,LLUVIA_legazpi
Fecha,,,,,,,,,,,,,,,,,,,,,
1999-03-16 17:00:00,1.1905,NaN,0.254916,-0.163239,0.282931,-0.018720,NaN,-0.172799,NaN,-0.175859,...,-0.197428,-0.177951,-0.164368,NaN,NaN,NaN,-0.16558,NaN,NaN,NaN
1999-03-16 18:00:00,1.1905,NaN,0.254916,-0.163239,0.282931,-0.018720,NaN,-0.172799,NaN,-0.175859,...,-0.197428,-0.177951,-0.164368,NaN,NaN,NaN,-0.16558,NaN,NaN,NaN
1999-03-16 19:00:00,1.1905,NaN,0.254916,-0.163239,0.282931,-0.018720,NaN,-0.172799,NaN,-0.175859,...,-0.197428,-0.177951,-0.164368,NaN,NaN,NaN,-0.16558,NaN,NaN,NaN
1999-03-16 20:00:00,1.1905,NaN,0.254916,-0.163239,0.282931,-0.018720,NaN,-0.172799,NaN,-0.175859,...,-0.197428,-0.177951,-0.164368,NaN,NaN,NaN,-0.16558,NaN,NaN,NaN
1999-03-16 21:00:00,1.1905,NaN,0.254916,-0.163239,0.265546,-0.029147,NaN,-0.172799,NaN,-0.175859,...,-0.197428,-0.177951,-0.164368,NaN,NaN,NaN,-0.16558,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-30 19:00:00,0.1553,-0.18246,-0.662379,-0.163239,NaN,NaN,-0.176252,-0.172799,-0.177317,-0.175859,...,-0.197428,-0.177951,-0.164368,-0.1841,-0.7878,-0.434755,-0.16558,-0.158125,-0.110789,-0.178171
2023-09-30 20:00:00,0.1553,-0.18246,-0.662379,-0.163239,NaN,NaN,-0.176252,-0.172799,-0.177317,-0.175859,...,-0.197428,-0.177951,-0.164368,-0.1841,-0.7878,-0.434755,-0.16558,-0.158125,-0.110789,-0.178171
2023-09-30 21:00:00,0.1489,-0.18246,-0.675483,-0.163239,NaN,NaN,-0.176252,-0.172799,-0.177317,-0.175859,...,-0.197428,-0.177951,-0.164368,-0.1841,-0.7878,-0.434755,-0.16558,-0.158125,-0.110789,-0.178171


In [9]:
minmax_scaler = MinMaxScaler(feature_range=(0, 1))
df = pd.DataFrame(minmax_scaler.fit_transform(df), columns=df.columns, index=df.index)
df

,CAUDAL_RIO_Aitzu,LLUVIA_Aitzu,NIVEL_RIO_Aitzu,LLUVIA_Aizarnazabal,NIVEL_RIO_Aizarnazabal,CAUDAL_RIO_Aizarnazabal,LLUVIA_Antzuola,LLUVIA_Arantzazu,LLUVIA_Araotz,LLUVIA_Arkaka,...,LLUVIA_Lareo,LLUVIA_Lastur,LLUVIA_Laurgain,LLUVIA_Matxinbenta,NIVEL_RIO_Matxinbenta,CAUDAL_RIO_Matxinbenta,LLUVIA_Troya,LLUVIA_zumarraga,LLUVIA_azpeitia,LLUVIA_legazpi
Fecha,,,,,,,,,,,,,,,,,,,,,
1999-03-16 17:00:00,0.013467,NaN,0.129105,0.0,0.137490,0.014634,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1999-03-16 18:00:00,0.013467,NaN,0.129105,0.0,0.137490,0.014634,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1999-03-16 19:00:00,0.013467,NaN,0.129105,0.0,0.137490,0.014634,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1999-03-16 20:00:00,0.013467,NaN,0.129105,0.0,0.137490,0.014634,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1999-03-16 21:00:00,0.013467,NaN,0.129105,0.0,0.136401,0.014357,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-30 19:00:00,0.001757,0.0,0.049830,0.0,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.07541,0.002149,0.0,0.0,0.0,0.0
2023-09-30 20:00:00,0.001757,0.0,0.049830,0.0,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.07541,0.002149,0.0,0.0,0.0,0.0
2023-09-30 21:00:00,0.001684,0.0,0.048698,0.0,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.07541,0.002149,0.0,0.0,0.0,0.0


In [10]:
df.columns

Index(['CAUDAL_RIO_Aitzu', 'LLUVIA_Aitzu', 'NIVEL_RIO_Aitzu',
       'LLUVIA_Aizarnazabal', 'NIVEL_RIO_Aizarnazabal',
       'CAUDAL_RIO_Aizarnazabal', 'LLUVIA_Antzuola', 'LLUVIA_Arantzazu',
       'LLUVIA_Araotz', 'LLUVIA_Arkaka', 'LLUVIA_Barrendiola',
       'NIVEL_RIO_Barrendiola', 'CAUDAL_RIO_Barrendiola',
       'LLUVIA_Barrendiola_Pluviometro', 'LLUVIA_Elosua', 'LLUVIA_Erdoizta',
       'LLUVIA_Ibaieder', 'NIVEL_RIO_Ibaieder', 'CAUDAL_RIO_Ibaieder',
       'LLUVIA_Lareo', 'LLUVIA_Lastur', 'LLUVIA_Laurgain',
       'LLUVIA_Matxinbenta', 'NIVEL_RIO_Matxinbenta', 'CAUDAL_RIO_Matxinbenta',
       'LLUVIA_Troya', 'LLUVIA_zumarraga', 'LLUVIA_azpeitia',
       'LLUVIA_legazpi'],
      dtype='object')

In [12]:
from collections import Counter

group_dict = {'CAUDAL_RIO_Aitzu': 1,
              'LLUVIA_Aitzu':2,
              'NIVEL_RIO_Aitzu':3,
              'LLUVIA_Aizarnazabal':2,
              'NIVEL_RIO_Aizarnazabal':3,
              'CAUDAL_RIO_Aizarnazabal':4, 
              'LLUVIA_Antzuola':2, 
              'LLUVIA_Arantzazu':2,
              'LLUVIA_Araotz':2, 
              'LLUVIA_Arkaka':2, 
              'LLUVIA_Barrendiola':2,
              'NIVEL_RIO_Barrendiola':3, 
              'CAUDAL_RIO_Barrendiola':4,
              'LLUVIA_Barrendiola_Pluviometro':2, 
              'LLUVIA_Elosua':2, 
              'LLUVIA_Erdoizta':2,
              'LLUVIA_Ibaieder':2, 
              'NIVEL_RIO_Ibaieder':3, 
              'CAUDAL_RIO_Ibaieder':4,
              'LLUVIA_Lareo':2, 
              'LLUVIA_Lastur':2, 
              'LLUVIA_Laurgain':2,
              'LLUVIA_Matxinbenta':2, 
              'NIVEL_RIO_Matxinbenta':3, 
              'CAUDAL_RIO_Matxinbenta':4,
              'LLUVIA_Troya':2, 
              'LLUVIA_zumarraga':2, 
              'LLUVIA_azpeitia':2,
              'LLUVIA_legazpi':2
    
}

df_group = df.groupby(by=group_dict, axis=1).mean()
df_group

,1,2,3,4
Fecha,,,,
1999-03-16 17:00:00,0.013467,0.0,0.120546,0.033981
1999-03-16 18:00:00,0.013467,0.0,0.120546,0.033981
1999-03-16 19:00:00,0.013467,0.0,0.120546,0.033981
1999-03-16 20:00:00,0.013467,0.0,0.120546,0.033981
1999-03-16 21:00:00,0.013467,0.0,0.120273,0.033889
...,...,...,...,...
2023-09-30 19:00:00,0.001757,0.0,0.049130,0.003043
2023-09-30 20:00:00,0.001757,0.0,0.049130,0.003043
2023-09-30 21:00:00,0.001684,0.0,0.048816,0.003018


In [13]:
scaled = df_group.values

In [14]:
scaled = np.nan_to_num(scaled, nan=-1000)

In [15]:
scaled

array([[0.01346697, 0.        , 0.12054569, 0.0339809 ],
       [0.01346697, 0.        , 0.12054569, 0.0339809 ],
       [0.01346697, 0.        , 0.12054569, 0.0339809 ],
       ...,
       [0.00168436, 0.        , 0.04881637, 0.00301793],
       [0.00175676, 0.        , 0.04906866, 0.00299451],
       [0.00168436, 0.        , 0.04878553, 0.00299451]])

In [16]:
backward_step = 10
forward_steps = 6
n_features = df_group.shape[1]

In [43]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]

    
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    if dropnan:
        agg.dropna(inplace=True)
    return agg
        

In [19]:
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        if out_end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [72]:
def create_X_Y (ts: np.array, lag=1, n_ahead=1, target_index=0) -> tuple:
    n_features = ts.shape[1]
    X, Y = [], []
    
    if len(ts) - lag <= 0:
        X.append(ts)
    else:
        for i in range(len(ts) - lag - n_ahead):
            Y.append(ts[(i + lag):(i + lag + n_ahead), target_index])
            X.append(ts[i:(i + lag)])
    
    X, Y = np.array(X), np.array(Y)
    X = np.reshape(X, (X.shape[0], lag, n_features))
    
    return X, Y

In [74]:
reframed = series_to_supervised(scaled, backward_step, forward_steps)

In [75]:
test_share = 1 - TRAIN_TEST_SPLIT_NUM

In [76]:
X, y = create_X_Y(scaled, lag=backward_step, n_ahead=forward_steps)
n_ft = X.shape[2]

In [77]:
test_X, test_y = X[0:int(X.shape[0] * (1 - test_share))], y[0:int(X.shape[0] * (1 - test_share))]
train_X, train_y = X[int(X.shape[0] * (1 - test_share)):], y[int(X.shape[0] * (1 - test_share)):]

In [78]:
def custom_cost_function(y_true, y_pred):
    weights = tf.where(y_true > y_pred, 0.5, 1.0)
    cost  =tf.reduce_mean(tf.square(y_true - y_pred) * weights)
    return cost

In [79]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(tf.keras.layers.LSTM(64, return_sequences=False))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(forward_steps, activation='linear'))
model.compile(loss='mae', optimizer='adam')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 10, 128)           68096     
                                                                 
 lstm_4 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 390       
                                                                 
Total params: 117894 (460.52 KB)
Trainable params: 117894 (460.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [80]:
print(train_X.shape)

(204371, 10, 4)


In [83]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=12, min_lr=0.00001, mode='min', verbose=1)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
callback = [reduce_lr, early_stopping]

history = model.fit(train_X, train_y, epochs=300, batch_size=1500, callbacks=[callback], validation_data=(test_X, test_y), verbose=1, shuffle=False)

plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.ylabel('mean absolute error')
plt.legend()
plt.show()

Epoch 1/300
137/137 [==============================] - 20s 149ms/step - loss: 10.1414 - val_loss: 0.0028 - lr: 0.0010
Epoch 2/300
 37/137 [=======>......................] - ETA: 15s - loss: 7.2369

KeyboardInterrupt: 

In [85]:
df = aitsu_pred


In [84]:
test_X.shape

(10756, 10, 4)

In [ ]:
yhat = model.predict(test_X)
pred_scaler = MinMaxScaler(feature_range=(0, 1)).fit(df.values[:,0].reshape(-1, 1))
inv_yhat = pred_scaler.inverse_transform(yhat)
inv_y = pred_scaler.inverse_transform(test_y)

In [ ]:
rmse = np.sqrt(mean_squared_error(abs(inv_y), abs(inv_yhat)))
print('Test RMSE: %.3f' % rmse)

In [ ]:
episodio = []
for i in inv_y[:]:
    episodio.append(i[0])

In [ ]:
episodio_pred = []
for i in inv_yhat:
    episodio_pred.append(i[0])

In [ ]:
rmse = np.sqrt(mean_squared_error(episodio, episodio_pred))
print('Test RMSE: %.3f' % rmse)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(episodio, episodio_pred, label = "scatter")
plt.plot(range(0,1200,1), label = "scatter2", color='r')
plt.legend()
plt.show()